In [1]:
# 4117351000 비산1동

SyntaxError: invalid syntax (<ipython-input-2-30828d50bf55>, line 1)

In [25]:
def make_tmfc(): # 현재 시간을 기점으로 가장 마지막 예보 시간을 리턴
    tmFc = time.strftime('%Y%m%d', time.localtime(time.time()))
    hour = int(time.strftime('%H', time.localtime(time.time())))
    if hour >= 18:
        tmFc+="1800"
    else:
        tmFc+="0600"
    return "&tmFc="+tmFc

In [26]:
import json
import urllib.request
import time

import json
with open("./config/config.json", "r") as sk_json:
    service_key = json.load(sk_json)['key']
    
class ShortWeatherService:
    
    def get_temperature(self,regID = '11B10101'):
        address = "http://apis.data.go.kr/1360000/VilageFcstMsgService/getLandFcst?serviceKey="+service_key+"&numOfRows=10&pageNo=1&numOfRows=10&pageNo=1&dataType=JSON"
        req = urllib.request.urlopen(address+"&regId="+regID)
        res = req.readline()
        print(address+"&regId="+regID)

        j = json.loads(res)

        if j['response']['header']['resultCode'] == '00':
            j = j['response']['body']['items']['item'][0]
            print(j)
        else :
            print('API CALL Failure')
            
    def get_fcst(self, regID = '11B00000'):
        address = "http://apis.data.go.kr/1360000/MidFcstInfoService/getMidLandFcst?serviceKey="+service_key+"&numOfRows=10&pageNo=1&dataType=JSON"
        req = urllib.request.urlopen(address+make_tmfc()+"&regId="+regID)
        print(address+make_tmfc()+"&regId="+regID)
        res = req.readline()
        j = json.loads(res)
        if j['response']['header']['resultCode'] == '00':
            j = j['response']['body']['items']['item'][0]
            print(j)
        else :
            print('API CALL Failure')

class MidWeatherService: # 중기예보 서비스 모듈
    #default = 서울

    
    def get_temperature(self,regID = '11B10101'):
        address = "http://apis.data.go.kr/1360000/MidFcstInfoService/getMidTa?serviceKey="+service_key+"&numOfRows=10&pageNo=1&dataType=JSON"
        req = urllib.request.urlopen(address+make_tmfc()+"&regId="+regID)
        res = req.readline()

        j = json.loads(res)

        if j['response']['header']['resultCode'] == '00':
            j = j['response']['body']['items']['item'][0]
            print(j)
        else :
            print('API CALL Failure')
            
    def get_fcst(self, regID = '11B00000'):
        address = "http://apis.data.go.kr/1360000/MidFcstInfoService/getMidLandFcst?serviceKey="+service_key+"&numOfRows=10&pageNo=1&dataType=JSON"
        req = urllib.request.urlopen(address+make_tmfc()+"&regId="+regID)
        print(address+make_tmfc()+"&regId="+regID)
        res = req.readline()
        j = json.loads(res)
        if j['response']['header']['resultCode'] == '00':
            j = j['response']['body']['items']['item'][0]
            print(j)
        else :
            print('API CALL Failure')
            


In [27]:
mid = MidWeatherService()
mid.get_temperature()

{'regId': '11B10101', 'taMin3': 21, 'taMin3Low': 1, 'taMin3High': 1, 'taMax3': 29, 'taMax3Low': 1, 'taMax3High': 1, 'taMin4': 22, 'taMin4Low': 1, 'taMin4High': 1, 'taMax4': 30, 'taMax4Low': 2, 'taMax4High': 2, 'taMin5': 22, 'taMin5Low': 1, 'taMin5High': 2, 'taMax5': 29, 'taMax5Low': 3, 'taMax5High': 1, 'taMin6': 22, 'taMin6Low': 1, 'taMin6High': 1, 'taMax6': 30, 'taMax6Low': 2, 'taMax6High': 1, 'taMin7': 21, 'taMin7Low': 2, 'taMin7High': 1, 'taMax7': 29, 'taMax7Low': 1, 'taMax7High': 1, 'taMin8': 21, 'taMin8Low': 0, 'taMin8High': 1, 'taMax8': 29, 'taMax8Low': 0, 'taMax8High': 1, 'taMin9': 22, 'taMin9Low': 0, 'taMin9High': 1, 'taMax9': 29, 'taMax9Low': 0, 'taMax9High': 1, 'taMin10': 22, 'taMin10Low': 0, 'taMin10High': 1, 'taMax10': 28, 'taMax10Low': 0, 'taMax10High': 2}


In [28]:
short = ShortWeatherService()

In [29]:
short.get_temperature()



http://apis.data.go.kr/1360000/VilageFcstMsgService/getLandFcst?serviceKey=0f2to44ZSBP527qwIw6lc6b0cRp22W2ThXgWnrfMGyXeaLSHv7fswO0997FTVjRP38E5jNoHB0m0mZZuNDQx%2BQ%3D%3D&numOfRows=10&pageNo=1&numOfRows=10&pageNo=1&dataType=JSON&regId=11B10101


JSONDecodeError: Expecting value: line 1 column 1 (char 0)